In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print("Text Length: {}".format(len(text)))

Text Length: 1115394


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char, _ in zip(char2idx, range(20)):
    print('    {:4s}: {:3d},'.format(repr(char),char2idx[char]))
print('    ...\n}')

{
    '\n':   0,
    ' ' :   1,
    '!' :   2,
    '$' :   3,
    '&' :   4,
    "'" :   5,
    ',' :   6,
    '-' :   7,
    '.' :   8,
    '3' :   9,
    ':' :  10,
    ';' :  11,
    '?' :  12,
    'A' :  13,
    'B' :  14,
    'C' :  15,
    'D' :  16,
    'E' :  17,
    'F' :  18,
    'G' :  19,
    ...
}


In [8]:
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
seq_length = 100
examples_per_epoch = len(text) // seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in dataset.take(1):
    print('Input Data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target Data: ', repr(''.join(idx2char[target_example.numpy()])))

Input Data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target Data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print('{:4d}step'.format(i))
    print("    Input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("    Expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0step
    Input: 18 ('F')
    Expected output: 47 ('i')
   1step
    Input: 47 ('i')
    Expected output: 56 ('r')
   2step
    Input: 56 ('r')
    Expected output: 57 ('s')
   3step
    Input: 57 ('s')
    Expected output: 58 ('t')
   4step
    Input: 58 ('t')
    Expected output: 1 (' ')


In [14]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [15]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [128]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [18]:
with tf.device('cpu:0'):
    for input_example_batch, target_example_batch in dataset.take(1):
        example_batch_predictions = model(input_example_batch)
        print(example_batch_predictions.shape, "# (Batch size, Sequence length, vocab size)")

(64, 100, 65) # (Batch size, Sequence length, vocab size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([50, 14, 39, 19, 57, 35, 40, 11, 42, 35, 24, 25,  3, 38, 33,  8, 62,
       61,  3,  4,  4, 63, 17, 42, 63, 48, 63, 56, 26, 35,  0, 56, 34, 56,
        8, 28, 14, 34,  4, 32,  8, 12, 16, 48, 54, 55, 33,  2, 59, 28,  5,
       59, 19, 43, 36, 32,  8,  0, 30, 11, 31, 63, 45, 59, 39, 35, 39, 64,
        8,  0, 17, 62, 14, 21, 17, 57, 54, 56, 63, 47, 39, 41, 28, 19, 49,
       64,  0, 12, 54, 32,  0, 47,  1, 16, 10, 16, 53, 60, 29, 16],
      dtype=int64)

In [23]:
print('Input: \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Predicted next char: \n', repr(''.join(idx2char[sampled_indices])))

Input: 
 "ke him\ndown, an a' were lustier than he is, and twenty such\nJacks; and if I cannot, I'll find those "

Predicted next char: 
 "lBaGsWb;dWLM$ZU.xw$&&yEdyjyrNW\nrVr.PBV&T.?DjpqU!uP'uGeXT.\nR;SyguaWaz.\nExBIEspryiacPGkz\n?pT\ni D:DovQD"


In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print('Predicted array shape: ', example_batch_predictions.shape, '#batch size, sequence length, vocab sizse')
print('Scalar Loss:           ', example_batch_loss.numpy().mean())

Predicted array shape:  (64, 100, 65) #batch size, sequence length, vocab sizse
Scalar Loss:            4.174472


In [129]:
model.compile(optimizer='adam', loss=loss)

In [130]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only=True
)

In [131]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 24s 139ms/step - loss: 2.5945
Epoch 2/10
172/172 [==============================] - 24s 141ms/step - loss: 1.8916
Epoch 3/10
172/172 [==============================] - 24s 142ms/step - loss: 1.6368
Epoch 4/10
172/172 [==============================] - 24s 142ms/step - loss: 1.5024
Epoch 5/10
172/172 [==============================] - 24s 142ms/step - loss: 1.4211
Epoch 6/10
172/172 [==============================] - 24s 142ms/step - loss: 1.3647
Epoch 7/10
172/172 [==============================] - 24s 142ms/step - loss: 1.3186
Epoch 8/10
172/172 [==============================] - 24s 142ms/step - loss: 1.2802
Epoch 9/10
172/172 [==============================] - 25s 143ms/step - loss: 1.2431
Epoch 10/10
172/172 [==============================] - 25s 143ms/step - loss: 1.2074


In [132]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [133]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [134]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_6 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [135]:
def generate_text(model, start_string):
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = 0.5
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [136]:
print(generate_text(model, start_string=u'ROMEO: '))

ROMEO: her part calls upon my soul,
To see his brave fellows to be compare.

GLOUCESTER:
And so I cannot seem the world as I have done,
And then the shore of all the heavens remember
The seal of this fair words and the treason.

MIRANDA:
O that is mine own brother, but I come to the
son?

DUKE VINCENTIO:
I have seen thee so far from him, but be bold
And bring it off the sease that can I have some service.

ANTONIO:
The people should he was and by and all this attemblance.

MENENIUS:
The poil of my sovereign lies the time;
And send his body to the second complexity,
Which with his father seem the streets of mine,
That I may live to be to say to him.

First Senator:
You shall not be the second feeling tears?

PETRUCHIO:
And what say you to the friar?

Lord Marshal:
The sight of the earth will care for me;
But what said 'sir, there is no more as things before the child,
On such a shrewd by the earld of my son.

KING RICHARD II:
Alack, indeed, you must be made a king?

PETRUCHIO:
A doy lor

In [137]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size = BATCH_SIZE
)

In [138]:
optimizer = tf.keras.optimizers.Adam()

In [139]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True
            )
        )
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    return loss

In [140]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = model.reset_states()
    
    for(batch_n, (inp,target)) in enumerate(dataset):
        loss = train_step(inp, target)
        
        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))
            
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
        
    print('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print('{} seconds per 1 Epoch'.format(time.time() - start))
    
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174795150756836
Epoch 1 Batch 100 Loss 2.3201682567596436
Epoch 1 Loss 2.0595
28.870805501937866 seconds per 1 Epoch
Epoch 2 Batch 0 Loss 2.0655455589294434
Epoch 2 Batch 100 Loss 1.8349605798721313
Epoch 2 Loss 1.7293
26.76218557357788 seconds per 1 Epoch
Epoch 3 Batch 0 Loss 1.7068895101547241
Epoch 3 Batch 100 Loss 1.5943655967712402
Epoch 3 Loss 1.5465
26.854443550109863 seconds per 1 Epoch
Epoch 4 Batch 0 Loss 1.5381665229797363
Epoch 4 Batch 100 Loss 1.4727193117141724
Epoch 4 Loss 1.4816
26.8249990940094 seconds per 1 Epoch
Epoch 5 Batch 0 Loss 1.4253158569335938
Epoch 5 Batch 100 Loss 1.4020183086395264
Epoch 5 Loss 1.3890
27.07475519180298 seconds per 1 Epoch
Epoch 6 Batch 0 Loss 1.3564867973327637
Epoch 6 Batch 100 Loss 1.3611595630645752
Epoch 6 Loss 1.3321
26.7163143157959 seconds per 1 Epoch
Epoch 7 Batch 0 Loss 1.2596545219421387
Epoch 7 Batch 100 Loss 1.2989153861999512
Epoch 7 Loss 1.3377
26.91620707511902 seconds per 1 Epoch
Epoch 8 Batch 0 Loss 